<a href="https://colab.research.google.com/github/wormulus/German-English-translator-seq2seq/blob/main/English_German_Translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import re
from google.colab import drive
drive.mount('/content/drive')

In [3]:
# Importing translations
data_path = "/content/drive/My Drive/Colab Notebooks/deu.txt"
# Defining lines as a list of each line
with open(data_path, 'r', encoding='utf-8') as f:
  lines = f.read().split('\n')

In [4]:
#@title Preprocessing
# Building empty lists to hold sentences
input_docs = []
target_docs = []
# Building empty vocabulary sets
input_tokens = set()
target_tokens = set()

# Adjust the number of lines for preprocessing
for line in lines[:2000]:
  # Input and target sentences are separated by tabs
  input_doc, target_doc = line.split('\t')[:2]
  # Appending each input sentence to input_docs
  input_docs.append(input_doc)

  target_doc = " ".join(re.findall(r"[\w']+|[^\s\w]", target_doc))
  # Redefine target_doc below
  # and append it to target_docs:
  target_doc = '<START> ' + target_doc + ' <END>'
  target_docs.append(target_doc)

  # Now we split up each sentence into words
  # and add each unique word to our vocabulary set
  for token in re.findall(r"[\w']+|[^\s\w]", input_doc):
    if token not in input_tokens:
      input_tokens.add(token)
  for token in target_doc.split():
    if token not in target_tokens:
      target_tokens.add(token)

input_tokens = sorted(list(input_tokens))
target_tokens = sorted(list(target_tokens))

# Create num_encoder_tokens and num_decoder_tokens:
num_encoder_tokens = len(input_tokens)
num_decoder_tokens = len(target_tokens)

max_encoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", input_doc)) for input_doc in input_docs])
max_decoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", target_doc)) for target_doc in target_docs])

input_features_dict = dict(
    [(token, i) for i, token in enumerate(input_tokens)])
target_features_dict = dict(
    [(token, i) for i, token in enumerate(target_tokens)])

reverse_input_features_dict = dict(
    (i, token) for token, i in input_features_dict.items())
reverse_target_features_dict = dict(
    (i, token) for token, i in target_features_dict.items())

encoder_input_data = np.zeros(
    (len(input_docs), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for line, (input_doc, target_doc) in enumerate(zip(input_docs, target_docs)):

  for timestep, token in enumerate(re.findall(r"[\w']+|[^\s\w]", input_doc)):
    # Assign 1. for the current line, timestep, & word
    # in encoder_input_data:
    encoder_input_data[line, timestep, input_features_dict[token]] = 1.

  for timestep, token in enumerate(target_doc.split()):

    decoder_input_data[line, timestep, target_features_dict[token]] = 1.
    if timestep > 0:

      decoder_target_data[line, timestep - 1, target_features_dict[token]] = 1.

# print out those value here to check on things:
print(list(input_features_dict.keys())[:50], reverse_target_features_dict[50])
print(input_tokens)


['!', '"', '$', ',', '.', '15', '18', '19', '3', '30', '45', '5', '7', '8', '9', ':', '?', 'A', 'Aim', 'All', 'Am', 'Answer', 'Are', 'Ask', 'Attack', 'Awesome', 'Back', 'Be', 'Beat', 'Begin', 'Birds', 'Bless', 'Bottoms', 'Bring', 'Burn', 'Bury', 'Buy', 'CD', 'Call', 'Calm', 'Can', 'Carry', 'Catch', 'Check', 'Cheer', 'Cheers', 'Chill', 'Choose', 'Come', 'Cool'] Beobachte
['!', '"', '$', ',', '.', '15', '18', '19', '3', '30', '45', '5', '7', '8', '9', ':', '?', 'A', 'Aim', 'All', 'Am', 'Answer', 'Are', 'Ask', 'Attack', 'Awesome', 'Back', 'Be', 'Beat', 'Begin', 'Birds', 'Bless', 'Bottoms', 'Bring', 'Burn', 'Bury', 'Buy', 'CD', 'Call', 'Calm', 'Can', 'Carry', 'Catch', 'Check', 'Cheer', 'Cheers', 'Chill', 'Choose', 'Come', 'Cool', 'Cuff', 'D', 'DJ', 'Definitely', 'Destroy', 'Did', 'Dig', 'Do', 'Dogs', "Don't", 'Drink', 'Drive', 'Drop', 'Duck', 'Duty', 'Eat', 'Examine', 'Excuse', 'Fantastic', 'Feel', 'Fill', 'Find', 'Fire', 'Fix', 'Fold', 'Follow', 'Forget', 'Forgive', 'Freeze', 'Get', 'Give

In [8]:
#@title Training Model
# Import tensorflow and keras layers
from tensorflow import keras
from keras.layers import Input, LSTM, Dense
from keras.models import Model

# to prevent mac errors. Can comment out if not needed
#import os
#os.environ['KMP_DUPLICATE_LIB_OK']='True'

# Choose a dimensionality
latent_dim = 256

# Choose a batch size
# and a number of epochs:
batch_size = 10
epochs = 75

# Encoder training setup
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_hidden, state_cell = encoder_lstm(encoder_inputs)
encoder_states = [state_hidden, state_cell]

# Decoder training setup:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, decoder_state_hidden, decoder_state_cell = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Building the training model:
training_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

print("Model summary:\n")
training_model.summary()
print("\n\n")

# Compile the model:
training_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# print("Training the model:\n")
# Train the model:
training_model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size = batch_size, epochs = epochs, validation_split = 0.2)

training_model.save('training_model.h5')

Model summary:

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, None, 742)]  0           []                               
                                                                                                  
 input_8 (InputLayer)           [(None, None, 1254)  0           []                               
                                ]                                                                 
                                                                                                  
 lstm_4 (LSTM)                  [(None, 256),        1022976     ['input_7[0][0]']                
                                 (None, 256),                                                     
                                 (None, 256)]                               

In [9]:
#@title Testing the Model
from keras.models import load_model
import numpy as np

training_model = load_model('training_model.h5')
encoder_inputs = training_model.input[0]
encoder_outputs, state_h_enc, state_c_enc = training_model.layers[2].output
encoder_states = [state_h_enc, state_c_enc]

encoder_model = Model(encoder_inputs, encoder_states)


decoder_state_input_hidden = Input(shape=(latent_dim,))
decoder_state_input_cell = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_hidden, decoder_state_input_cell]
decoder_outputs, state_hidden, state_cell = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_hidden, state_cell]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

def decode_sequence(test_input):
  # Encode the input as state vectors.
  states_value = encoder_model.predict(test_input)

  # Generate empty target sequence of length 1.
  target_seq = np.zeros((1, 1, num_decoder_tokens))
  # Populate the first token of target sequence with the start token.
  target_seq[0, 0, target_features_dict['<START>']] = 1.

  # Sampling loop for a batch of sequences
  # (to simplify, here we assume a batch of size 1).
  decoded_sentence = ''

  stop_condition = False
  while not stop_condition:
    # Run the decoder model to get possible 
    # output tokens (with probabilities) & states
    output_tokens, hidden_state, cell_state = decoder_model.predict(
      [target_seq] + states_value)

    # Choose token with highest probability
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_token = reverse_target_features_dict[sampled_token_index]
    decoded_sentence += " " + sampled_token

    # Exit condition: either hit max length
    # or find stop token.
    if (sampled_token == '<END>' or len(decoded_sentence) > max_decoder_seq_length):
      stop_condition = True

    # Update the target sequence (of length 1).
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, sampled_token_index] = 1.

    # Update states
    states_value = [hidden_state, cell_state]

  return decoded_sentence


# Test thing soout!
for seq_index in range(50):
  test_input = encoder_input_data[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(test_input)
  print('-')
  print('Input sentence:', input_docs[seq_index])
  print('Decoded sentence:', decoded_sentence)

1/1 [==============================] - 0s 18ms/step
-
Input sentence: Go.
Decoded sentence:  Geh . <END>
1/1 [==============================] - 0s 19ms/step
-
Input sentence: Hi.
Decoded sentence:  Grüß Gott ! <END>
1/1 [==============================] - 0s 18ms/step
-
Input sentence: Hi.
Decoded sentence:  Grüß Gott ! <END>
1/1 [==============================] - 0s 19ms/step
-
Input sentence: Run!
Decoded sentence:  Lauf ! <END>
1/1 [==============================] - 0s 19ms/step
-
Input sentence: Run.
Decoded sentence:  Lauf ! <END>
1/1 [==============================] - 0s 25ms/step
-
Input sentence: Wow!
Decoded sentence:  Donnerwetter !
1/1 [==============================] - 0s 22ms/step
-
Input sentence: Wow!
Decoded sentence:  Donnerwetter !
1/1 [==============================] - 0s 16ms/step
-
Input sentence: Duck!
Decoded sentence:  Kopf runter ! <END>
1/1 [==============================] - 0s 19ms/step
-
Input sentence: Fire!
Decoded sentence:  Feuer ! <END>
1/1 [============